In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder. \
        appName("spark_sql").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

23/03/31 08:13:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# to read the data into Spark DataFrames
df_green = spark.read.parquet('./data/processed/green/*/*')
df_yellow = spark.read.parquet('./data/processed/yellow/*/*')
df_fhv = spark.read.parquet('./data/processed/fhv/*')

In [ ]:
# Find out the common columns between the two dataframes
set(df_green.columns) & set(df_yellow.columns)

In [ ]:
# df_green.select('lpep_pickup_datetime', 'lpep_dropoff_datetime').show()
# df = spark.read.parquet('./data/raw/green_tripdata_2020-01.parquet')
# df.printSchema()

df_yellow.printSchema()

In [3]:
# Renaming the datetime columns
df_green = df_green \
        .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
        .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')


df_yellow = df_yellow\
        .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
        .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [4]:
# Preserve the column order in df_green
common_cols = []

for col in df_green.columns:
    if (col in set(df_yellow.columns)):
        common_cols.append(col)
        
        
common_cols

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [5]:
from pyspark.sql import functions as F

# select the common columns from both dataframe and join to form one table
df_green_proc = df_green\
            .select(common_cols)\
            .withColumn('service_type', F.lit('green')) # help us identify which table did the data come from


df_yellow_proc = df_yellow\
            .select(common_cols)\
            .withColumn('service_type', F.lit('yellow')) # help us identify which table did the data come from

In [6]:
df_trips_data = df_green_proc\
                .unionAll(df_yellow_proc)

In [7]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green|77144856|
|      yellow|55553400|
+------------+--------+



Spark don't recognise df_trips_data as a table.
Have to register the dataframe as a temptable before you can query it using Spark SQL.

In [ ]:
spark.sql("""

select * from df_trips_data limit 10;

""")

In [8]:
# register as temp table
df_trips_data.registerTempTable('trips_data')

In [ ]:
spark.sql("""

select * from trips_data limit 10;

""").show()

In [ ]:
# re-write the groupby statement using spark.sql
spark.sql("""

select
    service_type,
    count(1) as count
from trips_data
group by service_type;

""").show()

In [9]:
df_trips_data.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- service_type: string (nullable = false)



In [10]:
df_result = spark.sql("""

select
    -- revenue grouping
    PULocationID as revenue_zone,
    date_trunc('month', pickup_datetime) as revenue_month,
    service_type,
    
    -- revenue calculation
    sum(fare_amount) as revenue_monthly_fare,
    sum(extra) as revenue_monthly_extra,
    sum(mta_tax) as revenue_monthly_mta_tax,
    sum(tip_amount) as revenue_monthly_tip_amount,
    sum(tolls_amount) as revenue_monthly_tolls_amount,
    sum(improvement_surcharge) as revenue_monthly_improvement_surcharge,
    sum(total_amount) as revenue_monthly_total_amount,
    sum(congestion_surcharge) as revenue_monthly_congestion_surcharge,
    
    
    -- additional calculations
    avg(passenger_count) as avg_monthly_passenger_count,
    avg(trip_distance) as avg_monthly_trip_distance
    
from trips_data
group by 1,2,3
-- group by revenue_zone, revenue_month, service_type

""")

In [12]:
# this will generate a lot of small files
df_result.write.parquet('./data/report/revenue/')

In [14]:
# to reduce the number of partitions

no_of_partitions = 1
df_result.coalesce(no_of_partitions).write.parquet('./data/report/revenue/', mode = 'overwrite')

In [15]:
! ls -h ./data/report/revenue

_SUCCESS  part-00000-9e1cf35f-d9f7-451f-a0c3-85f27c8c981f-c000.snappy.parquet
